In [ ]:
# default_exp model.rnnvae

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# export

import torch
from torch import nn, optim
from deeptool.architecture import Encoder, Decoder, DownUpConv
from deeptool.utils import Tracker

# RNN VAE

> Structure for an Approach maintained a pseudo space realtion



In [ ]:
# load some test dataset to confirm architecture:
from deeptool.parameters import get_all_args
from deeptool.dataloader import load_test_batch
args = get_all_args()
args.model_type = "rnnvae"
args.batch_size = 1
args.track = False
batch = load_test_batch(args)
batch["img"].shape

torch.Size([1, 3, 16, 256, 256])

In [ ]:
# export


def mod_batch(batch, key="img"):
    """
    transform the batch to be compatible with the network by permuting
    """
    if len(batch[key].shape) > 4:
        batch[key] = batch[key][0, :, :, :, :]
        batch[key] = batch[key].permute(1, 0, 2, 3)
    return batch

In [ ]:
batchmod = mod_batch(batch)
batchmod["img"].shape

torch.Size([16, 3, 256, 256])

In [ ]:
# export


class RNN_AE(nn.Module):

    def __init__(self, device, args):
        """
        The recurrent autoencoder for compressing 3d data.
        It compresses in 2d while (hopefully) maintaining the spatial relation between layers
        """
        super(RNN_AE, self).__init__()
        self.device = device

        # 1. create the convolutional Encoder
        args.dim = 2
        self.conv_part_enc = DownUpConv(args, pic_size=args.pic_size, n_fea_in=len(
            args.perspectives), n_fea_next=args.n_fea_up, depth=1).to(self.device)

        # save important features
        max_fea, min_size = self.conv_part_enc.max_fea, self.conv_part_enc.min_size
        self.n_z, self.max_fea, self.min_size = args.n_z, max_fea, min_size

        self.view_arr = [-1, max_fea * min_size**2]  # as flat vector
        self.view_conv = [-1, max_fea, min_size, min_size]  # as conv block
        self.view_track = [1, len(args.perspectives), -1,
                           args.pic_size, args.pic_size]

        # 2. Apply FC- Encoder Part
        self.fc_part_enc = nn.Sequential(
            nn.Linear(max_fea*min_size*min_size, max_fea*min_size),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(max_fea*min_size, max_fea),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(max_fea, args.n_z),
        ).to(self.device)

        # 3. Transition Layer: GRU
        self.transition = nn.GRU(args.n_z, args.n_z, 1).to(self.device)

        # 4. Apply FC-Decoder Part
        self.fc_part_dec = nn.Sequential(
            nn.Linear(args.n_z, max_fea),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(max_fea, max_fea*min_size),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(max_fea*min_size, max_fea*min_size*min_size),
        ).to(self.device)

        # 5. create the convolutional Decoder
        self.conv_part_dec = DownUpConv(
            args, pic_size=args.pic_size, n_fea_in=len(
                args.perspectives), n_fea_next=args.n_fea_down, depth=1, move='up').to(self.device)

        # the standard loss
        self.mse_loss = nn.MSELoss(reduction='sum')

        # the optimizer
        self.optimizer = optim.Adam(self.parameters(), lr=args.lr)

        # reset the dimension
        args.dim = 3

        # Setup the tracker to visualize the progress
        if args.track:
            self.tracker = Tracker(args)

    def watch_progress(self, test_data, iteration):
        """Outsourced to Tracker"""
        self.tracker.track_progress(self, test_data, iteration)

    def rnn_transition(self, x):
        """
        take the matrix of encoded input slices and apply the RNN part
        """
        # reshape
        x = x.reshape([-1, 1, self.n_z])
        # apply GRU layer
        x, _ = self.transition(x)
        # reshape
        x = x.reshape([-1, self.n_z])
        return x

    def encode(self, x):
        x = self.conv_part_enc(x)
        x = x.reshape(self.view_arr)
        x = self.fc_part_enc(x)
        return x

    def decode(self, x):
        x = self.fc_part_dec(x)
        x = x.reshape(self.view_conv)
        x = self.conv_part_dec(x)
        return x
    
    def prep_input(self, batch):
        self.zero_grad()
        batch = mod_batch(batch)
        img = batch['img'].to(self.device)
        return img

    def forward(self, batch, update=True):
        """
        calculate the forward pass
        """
        # prepare
        img = self.prep_input(batch)
        # encode:
        x = self.encode(img)
        # apply the GRU transition
        x = self.rnn_transition(x)
        # decode
        x = self.decode(x)
        # calc loss
        loss = self.mse_loss(img, x)

        if update:
            loss.backward()
            self.optimizer.step()
            return x

        else:
            tr_data = {}
            tr_data["loss"] = loss.item()

        return x, tr_data

In [ ]:
# export


class RNN_VAE(RNN_AE):
    """
    inherit from RNN_AE and add the variational part
    """

    def __init__(self, device, args):
        #super(RNN_AE, self).__init__(device, args)
        RNN_AE.__init__(self, device, args)
        # 2. rewrite FC- Encoder Part
        max_fea, min_size = self.max_fea, self.min_size
        self.fc_part_enc = nn.Sequential(
            nn.Linear(max_fea*min_size*min_size, max_fea*min_size),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(max_fea*min_size, max_fea),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(max_fea, 2 * args.n_z),
        ).to(self.device)
        # get the kl facor
        self.beta = args.rnn_vae_beta
        
    def vae_sampling(self, x):
        mu, log_sig2 = x.chunk(2, dim=1)
        # get random matrix
        eps = torch.rand_like(
            mu, device=self.device)
        # sample together
        z = mu + torch.exp(torch.mul(0.5, log_sig2)) * eps
        return z, mu, log_sig2
    
    def kl_loss(self, mu, log_sig2):
        return -0.5 * torch.sum(1 - torch.pow(mu, 2) - torch.exp(log_sig2) + log_sig2)
        
    def forward(self, batch, update=True):
        # prepare
        img = self.prep_input(batch)
        # encode
        x = self.encode(img)
        # apply the vae sampling
        x, mu, log_sig2 = self.vae_sampling(x)
        # now the transition
        x = self.rnn_transition(x)
        # decode
        x = self.decode(x)
        
        # get loss
        ae_loss = self.mse_loss(img, x)
        vae_loss = self.kl_loss(mu, log_sig2)
        loss = ae_loss + self.beta * vae_loss
        
        if update:
            loss.backward()
            self.optimizer.step()
            return x
        
        else:
            tr_data = {}
            tr_data["loss"] = loss.item()
            tr_data["ae_loss"] = ae_loss.item()
            tr_data["vae_loss"] = vae_loss.item()

        return x, tr_data
        

In [ ]:
# export


class RNN_IN_VAE(RNN_VAE):
    """
    inherit from RNN_VAE and add the GAN part
    """

    def __init__(self, device, args):
        #super(RNN_AE, self).__init__(device, args)
        RNN_VAE.__init__(self, device, args)
        # add extra parameters
        self.alpha = args.alpha
        self.m = args.m
        self.n_pretrain = args.n_pretrain
        
    def forward(self, batch, update=True):
        
        # 1st Pass
        # prepare
        img = self.prep_input(batch)
        # encode
        x = self.encode(img)
        # apply the vae sampling
        z, mu, log_sig2 = self.vae_sampling(x)
        # now the transition
        x = self.rnn_transition(z)
        # decode
        x_re = self.decode(x)
        
        # get losses
        ae_loss = self.mse_loss(img, x_re)
        vae_loss = self.kl_loss(mu, log_sig2)
        
        # 2nd Pass
        x_re = self.encode(x_re.detach())
        z_re, mu_re, log_sig2_re = self.vae_sampling(x_re)
        
        # get losses
        vae_loss_2 = self.kl_loss(mu_re, log_sig2_re)
        
        
        
        
        loss = ae_loss + self.beta * vae_loss
        
        if update:
            loss.backward()
            self.optimizer.step()
            return x
        
        else:
            tr_data = {}
            tr_data["loss"] = loss.item()
            tr_data["ae_loss"] = ae_loss.item()
            tr_data["vae_loss"] = vae_loss.item()

        return x, tr_data
        

In [ ]:
# export 

def Creator_RNN_AE(device, args):
    """
    return an instance of the class depending on the mode set in args
    """
    if args.rnn_vae:
        model = RNN_VAE(device, args)
    else:
        model = RNN_VAE(device, args)
    return model

In [ ]:
batch["img"].shape

torch.Size([16, 3, 256, 256])

In [ ]:
device = torch.device("cuda:0" if (
        torch.cuda.is_available() and args.n_gpu > 0) else "cpu")
rnn_vae = RNN_VAE(device, args)
data = load_test_batch(args)
x, tr = rnn_vae(data, update=False)
x.shape

torch.Size([16, 3, 256, 256])

In [ ]:
batch = load_test_batch(args)
batch["img"].shape

torch.Size([1, 3, 16, 256, 256])

In [ ]:
# hide
from deeptool.train_loop import test_one_batch
from deeptool.parameters import get_all_args, compat_args
args = get_all_args()
args.pic_size = 32

In [ ]:
# 3 dim test
args.model_type = "rnnvae"
args.dim = 3
args = compat_args(args)
test_one_batch(args)

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_dataloader.ipynb.
Converted 01_architecture.ipynb.
Converted 02_utils.ipynb.
Converted 03_parameters.ipynb.
Converted 04_train_loop.ipynb.
Converted 10_diagnosis.ipynb.
Converted 20_dcgan.ipynb.
Converted 21_introvae.ipynb.
Converted 22_vqvae.ipynb.
Converted 23_rnn_vae.ipynb.
Converted 99_index.ipynb.
